In [ ]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd

In [ ]:
tf.__version__

In [ ]:
train_dir = r"train"
test_dir = r"test"
os.path.exists(train_dir)

In [ ]:
df = pd.read_csv('Training_set.csv')
df.shape

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory=train_dir,
    labels=None,
    batch_size=67,
    shuffle=True,
    color_mode='rgb',
    image_size = (128,128)
)
test_ds = tf.keras.utils.image_dataset_from_directory(
    directory=test_dir,
    labels=None,
    batch_size=67,
    shuffle=True,
    color_mode='rgb',
    image_size = (128,128)
)


In [ ]:
lmao = next(iter(train_ds))
lmao

In [ ]:
uniq = np.array(df['label'].unique())

In [ ]:
for i in range (uniq.size):
    uniq[i] = i

In [ ]:
unique = np.array(df['label'].unique())

In [ ]:
uniqi = dict(zip(unique, uniq))
uniqi

In [ ]:
uniqi = dict(zip(unique, uniq))
label_arr = []
for item in df['label']:
    label_arr.append(uniqi[item])
label_arr = tf.convert_to_tensor(label_arr)
label_arr

In [ ]:

# train_ds_v2 = tf.keras.utils.image_dataset_from_directory(
#     directory=train_dir,
#     labels=label_arr,
#     label_mode="int",
#     batch_size=67,
#     shuffle=True,
#     color_mode='rgb',
#     image_size = (128,128)
# )

In [ ]:
cur = os.getcwd()
testFol = "images_categorize"

os.mkdir(os.path.join(cur, testFol))
os.chdir('c:\\Users\\Tran Vu\\OneDrive\\Documents\\TestingPrj\\butterfly_classification\\images_categorize')
os.getcwd()

for labels in uniqi.keys():
    os.mkdir(labels)